# LCPB 23-24 exercise 1 (Convolutional Neural Network, CNN)
- Andrea Semenzato 2130973
- Pietro Bernardi 2097494
- Tomàs Mezquita 2109239
- Mariam Chokheli 2122278

In [52]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

## PART 1 - Exercise

### Point 1
The lesson discussed two CNN versions: one ending with a global max pooling layer (G), and one ending with a dense layer (D). Choose one of the two G and D versions for the remaining exercise, motivating your choice.

The <b>D</b> model's results are as follows:

<table>
    <tr>
        <td><img src="Model1AcLs.png" width="600"></td>
        <td><img src="Model1CM.png" width="400"></td>
    </tr>
</table>

The first plot starting from the left shows the training and validation accuracies as a function of the number of epochs. The plot clearly shows that the model starts to overfit for $N > 50$. The second plot shows the losses and the third plot shows the confusion matrix.

Conversely, for the <b>G</b> model's results are as follows:

<table>
    <tr>
        <td><img src="Model2AcLs.png" width="600"></td>
        <td><img src="Model2CM.png" width="400"></td>
    </tr>
</table>

The <b>G</b> model does not show overfitting across the whole number of epoch's range. Also, this model tends to show higher accuracy and lower loss thus it is the model that will be used for the remaining points of this exercise.

### Point 2
Try different optimizers, including at least Adam, RMSprop, and Nesterov. Each one should be tested for at least five values of the learning rate η (for example, including 10-5, 10-4, 10-3). The range could be different for different algorithms. Moreover, at least five fits per model should be collected to obtain statistics with an average and standard deviation of the model’s validation accuracy. Plot all results in a single panel of a figure as a function of η.

In [62]:
# Data
etas = [1e-5,5e-5,1e-4,1e-3,1e-2,5e-2]
accuracies = {
    'Adam':[0.5585, 0.7267, 0.7227, 0.7710, 0.7644, 0.7765],
    'SGD':[0.4279,0.6331,0.6165,0.7492,0.7581,0.7569],
    'RMSprop':[0.4321,0.7156,0.7310,0.7810,0.7521,0.3338],
    'Nadam':[0.4525,0.7283,0.7079,0.7548,0.7602,0.7633]
}
errors = {
    'Adam':[0.0887,0.0444,0.0426,0.0036,0.0051,0.0082],
    'SGD':[0.0129,0.0527,0.0622,0.0167,0.0049,0.0063],
    'RMSprop':[0.0657,0.0589,0.0399,0.0033,0.0041,0.0000],
    'Nadam':[0.0745,0.0427,0.0360,0.0042,0.0068,0.0078]
}

# Plots
fig_1 = plt.figure(figsize=(8,5))
ax0_fig1 = plt.subplot2grid((1,1), loc=(0,0), colspan=1, rowspan=1, fig=fig_1)
for mod_name in accuracies:
    ax0_fig1.errorbar(etas, accuracies[mod_name], yerr=errors[mod_name], label=mod_name, marker='.', elinewidth=1, capsize=4, markersize=10)
    diff_m = np.array(accuracies[mod_name])-np.array(errors[mod_name])
    diff_p = np.array(accuracies[mod_name])+np.array(errors[mod_name])
    #ax0_fig1.fill_between(etas, diff_m, diff_p, alpha=0.1)
ax0_fig1.set_xscale('log')
ax0_fig1.legend()
ax0_fig1.grid(True, which='both', linestyle='--', linewidth=0.5)
#ax0_fig1.set_yticks(fontsize=12)
ax0_fig1.set_title("Validation Accuracies across different Learning Rates", fontsize=12)#, fontweight='bold')
ax0_fig1.set_ylabel("Validation Accuracies", fontsize=12)
ax0_fig1.set_xlabel("Learning rate $\eta$", fontsize=12)
fig_1.tight_layout()

For each probed value of $\eta$, six fits of the same <b>G</b> model were executed in order to obtain sufficient statistics to assign an error to each accuracy value. As we can see, the four tested optimizers are fairly compatible with each other when $\eta_3=10^{-3}$ and $\eta_4=10^{-2}$. We also point out that during an earlier run, the region between $\eta_3$ and $\eta_4$ was probed and a similar behavior was observed.

We decided to choose the <b>RMSprop</b> optimizer since at $\eta = 10^{-3}$ it presented both the highest accuracy and the lowest standard deviation, indicating more consistent performance during each trial fit.

The first hyperparameter found is thus: $\eta = 10^{-3}$. The other parameters have yet to be modified at this point, so the kernel size $KS = 11$ and the number of filters in the first layer $NF = 5$.

The following figure shows the accuracies and loss for the best model. As we can see, the model tends to overfit for number of epochs $> 100$ but we expect to correct this problem after we introduce regularization.

<img src="bestModel_ACC_0.png" width=600>

### Point 3
Choose one among the best CNNs in step 2 (there could be equivalent ones within error bars). Using that CNN, try to understand the hidden patterns. Introduce regularization and study the kernels of the first layer, also by varying their length and their number (respectively, KS and NF in the notebook). Report your findings, including eventually improved validation accuracy, thanks to the regularization. However, note that this point’s scope differs from optimizing the validation accuracy: it focuses on interpreting the kernel shapes.

In the following, we consider the first layer. In the figure below we firstly show what the weights look like for the worst performing model:

<img src="worstModel.png" width=600>

and we compare it instead with the best performing model that was just selected:

<img src="bestModel_0.png" width=600>

Regularization is still not applied, but in the best performing model the weights appear to follow a pattern: their absolute value is greater near the edges of the kernel, reflecting the hidden patterns in the data. If that wasn't the case, the training procedure would not have been able to adjust the weights in such manner, as the worst model clearly displays.

We now introduce regularization and we firstly don't modify the other parameters in order to check the effect that L1 and L2 regularizers have on the first layer's kernels.

The following figure shows the effect of the L1 regularizer with $\lambda = 0.01$.

<img src="reg_init_L1.png" width=600>

The following figure shows instead the effect of the L2 regularizer with $\lambda = 0.01$.

<img src="reg_init_L2.png" width=600>

We see how especially in the case of L1 regularizer, the majority of the "central" weights tend to be flattened to zero with respect to the ones near the edges.

We now vary both the regularizer's $\lambda$ and the length and number of kernels by trying different combinations of the hyperparameters. We conducted the search with the following ranges:

| Hyperparameter | Values |
| --- | --- |
| Kernel Size | $$5, 7, 9, 11$$ |
| N. of filters | $$5, 7, 9, 11$$ |
| Regularizer $\lambda$ | $$0,10^{-5},10^{-4},10^{-3},10^{-2},10^{-1}$$ |

Also, we introduced, separately, both <b>L1</b> and <b>L2</b> regularization in order to compare the results. Out of all the possible combinations, we selected a few that showed a slight improvement in validation accuracy and we then proceeded to test those more thoroughly executing multiple fits for each one. These are as follows:

<table>
    <tr><td><b>L1 regularized model's accuracies</b></td><td><b>L2 regularized model's accuracies</b></td></tr>
    <tr>
        <td>
            <table>
                <tr>
                    <td><b>#</b></td>
                    <td><b>Kernel Size</b></td>
                    <td><b>Number of Filters</b></td>
                    <td><b>Reg. $\lambda$</b></td>
                    <td><b>L1 accuracy</b></td>
                    <td><b>L1 accuracy $\sigma$</b></td>
                </tr>
                <tr>
                    <td>0</td>
                    <td>9</td>
                    <td>9</td>
                    <td>0.01</td>
                    <td>0.7940</td>
                    <td>0.0032</td>
                </tr>
                <tr>
                    <td>1</td>
                    <td>7</td>
                    <td>11</td>
                    <td>0.01</td>
                    <td>0.7898</td>
                    <td>0.0015</td>
                </tr>
                <tr>
                    <td>2</td>
                    <td>7</td>
                    <td>9</td>
                    <td>0.0001</td>
                    <td>0.7815</td>
                    <td>0.0029</td>
                </tr>
                <tr>
                    <td>3</td>
                    <td>9</td>
                    <td>5</td>
                    <td>0.01</td>
                    <td>0.7770</td>
                    <td>0.0103</td>
                </tr>
                <tr>
                    <td>4</td>
                    <td>9</td>
                    <td>7</td>
                    <td>0.001</td>
                    <td>0.7760</td>
                    <td>0.0046</td>
                </tr>
                <tr>
                    <td>5</td>
                    <td>11</td>
                    <td>7</td>
                    <td>0.01</td>
                    <td>0.7760</td>
                    <td>0.0065</td>
                </tr>
                <tr>
                    <td>6</td>
                    <td>11</td>
                    <td>5</td>
                    <td>0.01</td>
                    <td>0.7655</td>
                    <td>0.0044</td>
                </tr>
                <tr>
                    <td>7</td>
                    <td>11</td>
                    <td>9</td>
                    <td>0.001</td>
                    <td>0.7505</td>
                    <td>0.0076</td>
                </tr>
                <tr>
                    <td>8</td>
                    <td>11</td>
                    <td>11</td>
                    <td>0.001</td>
                    <td>0.7123</td>
                    <td>0.0044</td>
                </tr>
            </table>
        </td>
        <td>
            <table>
                <tr>
                    <td><b>#</b></td>
                    <td><b>Kernel Size</b></td>
                    <td><b>Number of Filters</b></td>
                    <td><b>Reg. $\lambda$</b></td>
                    <td><b>L2 accuracy</b></td>
                    <td><b>L2 accuracy $\sigma$</b></td>
                </tr>
                <tr>
                    <td>0</td>
                    <td>9</td>
                    <td>9</td>
                    <td>0.01</td>
                    <td>0.7848</td>
                    <td>0.0050</td>
                </tr>
                <tr>
                    <td>1</td>
                    <td>9</td>
                    <td>7</td>
                    <td>0.001</td>
                    <td>0.7815</td>
                    <td>0.0054</td>
                </tr>
                <tr>
                    <td>2</td>
                    <td>9</td>
                    <td>5</td>
                    <td>0.01</td>
                    <td>0.7802</td>
                    <td>0.0020</td>
                </tr>
                <tr>
                    <td>3</td>
                    <td>11</td>
                    <td>9</td>
                    <td>0.001</td>
                    <td>0.7752</td>
                    <td>0.0057</td>
                </tr>
                <tr>
                    <td>4</td>
                    <td>11</td>
                    <td>7</td>
                    <td>0.01</td>
                    <td>0.7635</td>
                    <td>0.0037</td>
                </tr>
                <tr>
                    <td>5</td>
                    <td>11</td>
                    <td>5</td>
                    <td>0.01</td>
                    <td>0.7633</td>
                    <td>0.0032</td>
                </tr>
                <tr>
                    <td>6</td>
                    <td>9</td>
                    <td>7</td>
                    <td>0.001</td>
                    <td>0.7620</td>
                    <td>0.0020</td>
                </tr>
                <tr>
                    <td>7</td>
                    <td>7</td>
                    <td>11</td>
                    <td>0.01</td>
                    <td>0.7420</td>
                    <td>0.0039</td>
                </tr>
                <tr>
                    <td>8</td>
                    <td>7</td>
                    <td>9</td>
                    <td>0.0001</td>
                    <td>0.6905</td>
                    <td>0.0061</td>
                </tr>
            </table>
        </td>
    </tr>
    
</table>

The best configuration was found to be the one reported on the first row of each table: $KS = 9, NF=9, \lambda=10^{-2}$ both for L1 and L2 regularizers.

The weights for the first layer with <b>L1</b> regularizer and the optimal set of hyperparameters are shown below:

<img src="reg_L1_final.png" width=600>

The weights for the first layer with <b>L2</b> regularizer and the optimal set of hyperparameters are shown below:

<img src="reg_L2_final.png" width=600>

When the L1 regularizer is used, the weights tends to be strongly pushed towards zero in the middle of the kernel while they are, for the most part, in absolute value, greater than zero only near the edges.

We now turn to the accuracies to see if and how the regularizers had made any impact on them.

The effect of the <b>L1</b> regularizer is shown below. The overfitting problem seems to have been greatly reduced with respect to the non-regularized version.

<table>
    <tr>
        <td>
            <img src="bestModel_ACC_L1.png" width="600">
        </td>
        <td>
            <img src="ConfMatrixFinal.png" width="400">
        </td>
    </tr>
</table>

On the contrary, the <b>L2</b> regularizer doesn't seem to produce much of an improvement since the model is still overfitting after regularization.
<table>
    <tr>
        <td>
            <img src="bestModel_ACC_L2.png" width="600">
        </td>
        <td>
            <img src="ConfMatrixFinal_L2.png" width="400">
        </td>
    </tr>
</table>

### Point 4 - OPTIONAL
Try to improve the CNN's performance by a random search in the hyperparameters' space. The performance is quantified again by the accuracy of the validation dataset.

We used the <b>RandomSearch</b> tuner[1], searching for optimal values of $\eta, KS$ and $NF$ both with L1 and L2 regularizers keeping $\lambda = 0.01$. The tuner reported the following optimal sets of hyperparameters:

1. For L1: $KS = 9, NF = 7, \eta = 0.01$
2. For L2: $KS = 9, NF = 9, \eta = 0.01$

The figure below shows the weights in the first layer for the L1 case (best accuracy was reported by the tuner as $0.7980$):

<img src="kt_output_L1.png" width="600">

The figure below shows the weights in the first layer for the L2 case (best accuracy was reported by the tuner as $0.7755$):

<img src="kt_output_L2.png" width="600">

## PART 2 - Code Appendix

### Keras Tuner Model Builder function

## Bibliography
[1] Introduction to the keras tuner (no date) TensorFlow. Available at: https://www.tensorflow.org/tutorials/keras/keras_tuner (Accessed: 09 March 2024). 